# Summary

```Objective```

todo

```Methods```

todo

```Results```

todo

# Config

# Libs

In [1]:
%load_ext autoreload
%autoreload 2
from misalign.models.components.autoencoder_kl import Encoder, AutoencoderKL
import torch
from misalign.models.components.proposed import VarAutoencoder_Loss
from misalign.models.components.perceptual import *


# Analysis

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

num_res_blocks = [2,2]
num_channels = [32, 64]
encoder = Encoder(
    in_channels=1,
    num_channels=[32, 64],
    out_channels=64,
    num_res_blocks=[2,2]
)  # You need to define the Encoder class somewhere
# Load the state dict
encoder.load_state_dict(torch.load("/home/kanghyun/misalign-benchmark/weights/encoder_pretrain_0.01.pth"))
encoder = encoder.to(device)

Using cuda


In [6]:
im = torch.ones(1,1,256,256).to(device)

with torch.no_grad():
    _, out = encoder(im)
print(out[0].shape, out[1].shape)

torch.Size([1, 32, 128, 128]) torch.Size([1, 64, 64, 64])


In [19]:
class localPerceptualLoss(RadImageNetPerceptualSimilarity):
    def forward(self, input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        We expect that the input is normalised between [0, 1]. Given the preprocessing performed during the training at
        https://github.com/BMEII-AI/RadImageNet, we make sure that the input and target have 3 channels, reorder it from
         'RGB' to 'BGR', and then remove the mean components of each input data channel. The outputs are normalised
        across the channels, and we obtain the mean from the spatial dimensions (similar approach to the lpips package).
        """
        # If input has just 1 channel, repeat channel to have 3 channels
        if input.shape[1] == 1 and target.shape[1] == 1:
            input = input.repeat(1, 3, 1, 1)
            target = target.repeat(1, 3, 1, 1)

        # Change order from 'RGB' to 'BGR'
        input = input[:, [2, 1, 0], ...]
        target = target[:, [2, 1, 0], ...]

        # Subtract mean used during training
        input = subtract_mean(input)
        target = subtract_mean(target)

        # Get model outputs
        outs_input = self.model.forward(input)
        outs_target = self.model.forward(target)

        # Normalise through the channels
        feats_input = normalize_tensor(outs_input)
        feats_target = normalize_tensor(outs_target)

        results = (feats_input - feats_target) ** 2

        return results

In [20]:
perceptual_loss = localPerceptualLoss().to(device)

In [21]:
loss = perceptual_loss(im, im)
print(loss.shape)

torch.Size([1, 2048, 8, 8])


In [10]:
loss

tensor(0., device='cuda:0')

In [12]:
loss_function = VarAutoencoder_Loss(crop_quarter=False).cuda()

In [13]:
loss = loss_function(im, im)

In [15]:
i = torch.zeros((1,64*64), device=im.device, requires_grad=True)
print(i.shape)

torch.Size([1, 4096])


In [9]:
loss[0,0]

tensor(0.2008, device='cuda:0')

In [10]:
loss[1,0]

tensor(0.2008, device='cuda:0')

In [11]:
# unfold

feature_im = torch.ones((1,32,64,64))
N, C, H, W = feature_im.shape
patch_size = 8

In [12]:
patch = torch.nn.functional.unfold(feature_im, kernel_size=patch_size, stride=1)
num_patches = patch.shape[-1]
patch = patch.view(N, C, patch_size, patch_size, num_patches)

print(patch.shape)

torch.Size([1, 32, 8, 8, 3249])


In [13]:
import numpy as np
np.sqrt(3249)

57.0

32.0